# Step 1 - Loading the data from our pipeline outputs for exploration
The code below will install the necessary dependencies in order to get started exploring the full dataset post data ingestion pipeline. You will need to run this first before the data is loaded into the notebook. 

In [ ]:
!pip install gdown
!pip install pyarrow

In [ ]:
import gdown

# Location of publicly available combined set for analysis post data ingestion pipeline run
url = "https://drive.google.com/drive/folders/1tUcqzoeM9AaDaUexcHg-OqXJeL6f5k6U?usp=share_link"
gdown.download_folder(url, quiet=True, use_cookies=False)

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

combined_data = pq.read_table(source="who_ears_social_listening_public/merged_owid_who_ears.parquet").to_pandas()
# We will need to reformat the date column for analysis
combined_data['date'] = pd.to_datetime(combined_data['date'], format = '%Y-%m-%d')

In [ ]:
combined_data.columns

In [ ]:
# This will show the first 10 columns and rows we just want to make sure we loaded the data. 
combined_data.head(10)

# Step 2 - Creating a Space for Data Exploration 
We want to give the ability for our Domain expert to explore all of the variables in order to determine the most interesting to answer the research question. Leveraging tools like dtale make it easy to examine things such as missing variable and correlations. 


In [ ]:
# Install the dtale package
# Note: You may have to restart the runtime in which case be sure and import the dataframe again from the above code. 
!pip install -U dtale
!pip install statsmodels --upgrade

In [ ]:
# lets examine out dataset 
import pandas as pd

import dtale
import dtale.app as dtale_app

dtale_app.USE_COLAB = True # Comment this out if you are using another environment

dtale.show(combined_data)

Follow the link above and lets analyze our data and select the right features! You might also want to get acquainted with the dtale documentation here https://github.com/man-group/dtale. 

# Step 3 - Feature Engineering - Extraction and Transformation
Now that we know which columns we want to extract we can pull these out. We have some clever insights from our domain expert who has instucted us to perform the following transformations on the variables in preparation for the time series model analysis. Once we experiment with this and we feel comfortable we are on the right track we can build this in the pipeline in prepartion for production. 

Tranform Step 1 - Create a dataframe with the following columns: 

* date
* mis_and_disinformation
* mis_and_disinformation_male
* mis_and_disinformation_female
* myths
* myths_female
* myths_male
* new_vaccinations_smoothed

Tranform Step 2 - Merge all myths and mis_disinformation for all columns respectively by summing the columns. The new columns names will will be prefixed with 'mis_myths_' (e.g. 'mis_myths_male', 'mis_myths_female' etc.) 

In [ ]:
# lets next create a sensible feature set for training and testing 
features = combined_data[['date', 'mis_and_disinformation', 'mis_and_disinformation_male',  
                          'mis_and_disinformation_female',
                         'myths','myths_female', 'myths_male', 'new_vaccinations_smoothed']]

# remove any NaN
features = features.dropna()

# Sum all the variables and rename
features = features.eval("mis_myths = myths + mis_and_disinformation")
features = features.eval("mis_myths_male = myths_male + mis_and_disinformation_male")
features = features.eval("mis_myths_female = myths_female + mis_and_disinformation_female")

# Lets drop the old columns now that we have merged the two into our new columns

features = features.drop(columns=['mis_and_disinformation', 'mis_and_disinformation_male',  
                          'mis_and_disinformation_female',
                         'myths','myths_female', 'myths_male'])

features.head(10)

# Step 4 - Selecting the optimal model for prediction 
Since this is time series data and we can reasonaly infer starting with analysis of performance on linear models. As we don't want to spend time writing boilerplate code lets start with a library that simply uses generic hyperparameters for a multitude of regression models and displays the results for us. 


In [ ]:
# Sadly there are a lot of dependencies that conflict with dtale so we need to remove them and install pycaret
!pip uninstall -y dtale
!pip uninstall -y statsmodels
# install pycaret as a way to do some quick analysis of all regression model types to know which performs the best 
!pip install --pre pycaret


In [ ]:
from pycaret.regression import *
# lets setup our first session for eval
# we chose total cases since its a complete set AND is has high correlation with the other features (Pearson)
# our X is all the data from the who-ears social media listening set and our Y 
# or target is the new_vaccinations_smoothed to predict

session_1 = setup(features, target = 'new_vaccinations_smoothed', 
                  session_id=1, 
                  log_experiment=False, 
                  experiment_name='new_vaccinations_smoothed_1')

In [ ]:
# We can now compare all the regression models available 
best_model = compare_models()